In [ ]:
# STEAMDB내 INDIEMODA LIST 파일 병합
import json
import pandas as pd

# 1) 두 JSON 파일 읽기
file1 = r"C:\Users\02105\dabootcamp\T1\Steam ApI 장르별 리뷰\steamdbjson\steamdb_indie_moba_1741935981.json"
file2 = r"C:\Users\02105\dabootcamp\T1\Steam ApI 장르별 리뷰\steamdbjson\steamdb_indie_moba_rest_of_pages.json"

with open(file1, "r", encoding="utf-8") as f:
    data1 = json.load(f)  # list of dict
with open(file2, "r", encoding="utf-8") as f:
    data2 = json.load(f)  # list of dict

# 2) 리스트 병합
merged_list = data1 + data2  # 단순 합치기

# (옵션) 3) 중복 제거
#  - 예: app_id 키를 기준으로 중복 판별
#  - 만약 "objectID"라는 키가 있다면 해당 키로 바꿔도 됨
unique_map = {}
for item in merged_list:
    key = item.get("app_id") or item.get("objectID")
    unique_map[key] = item

final_list = list(unique_map.values())
print(f"최종 항목 수: {len(final_list)}")

# 4) 합쳐진 리스트를 JSON으로 저장
merged_json_name = "steamdb_indie_moba_merged.json"
with open(merged_json_name, "w", encoding="utf-8") as f:
    json.dump(final_list, f, ensure_ascii=False, indent=4)
print(f"[DONE] {merged_json_name} 에 병합된 JSON 저장 완료.")

# 5) CSV로 저장
#   - pandas를 사용해 DataFrame으로 변환 후 to_csv
df = pd.DataFrame(final_list)
merged_csv_name = "steamdb_indie_moba_merged.csv"
df.to_csv(merged_csv_name, index=False, encoding="utf-8")
print(f"[DONE] {merged_csv_name} 에 CSV 저장 완료.")


최종 항목 수: 206
[DONE] steamdb_indie_moba_merged.json 에 병합된 JSON 저장 완료.
[DONE] steamdb_indie_moba_merged.csv 에 CSV 저장 완료.


In [2]:
import pandas as pd

# 병합된 JSON 파일 경로
merged_json = "steamdb_indie_moba_merged.json"

# 1) JSON → DataFrame
#   만약 JSON이 list of dicts 형태(records)라면:
df_merged = pd.read_json(merged_json, orient="records", encoding="utf-8")

# 2) 간단히 데이터 확인
print("\n=== HEAD ===")
print(df_merged.head())

print("\n=== INFO ===")
print(df_merged.info())

print("\n=== DESCRIBE ===")
print(df_merged.describe())

print("\n=== NULL CHECK ===")
print(df_merged.isnull().sum())

# (옵션) 전처리
# df_merged.dropna(inplace=True)
# if "price_us" in df_merged.columns:
#     df_merged["price_us"] = pd.to_numeric(df_merged["price_us"], errors="coerce")

print("[DONE] JSON 데이터 확인 완료. DB 업로드 전 점검 끝!")


=== HEAD ===
                                           name   app_id  \
0  Bishojo Mangekyo Kotowari to Meikyu no Shojo  1310990   
1                                      MageQuit   572220   
2                                    Spellsworn   360620   
3                                          惊险极速  1777280   
4                    Awesomenauts - the 2D moba   204300   

                                         oslist  price_us releaseYear  \
0             [Windows, Steam Deck Unsupported]     29.99        2021   
1  [Windows, macOS, Linux, Steam Deck Verified]     14.99        2019   
2                              [Windows, Linux]      0.00        2018   
3                                     [Windows]      6.99        2021   
4  [Windows, macOS, Linux, Steam Deck Playable]      0.00        2012   

   userScore  
0      91.93  
1      88.65  
2      84.85  
3      83.57  
4      83.02  

=== INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data colum

In [9]:
# 다국어 게임 네이밍으로 인한 오류로 코드 수정
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import VARCHAR, BIGINT, FLOAT
from dotenv import load_dotenv

# .env 파일에서 DB 연결 정보를 가져옵니다.
load_dotenv()
dbuser = os.getenv('dbuser')
password = os.getenv('password')
host = os.getenv('host')
port = os.getenv('port', 3306)  # 기본 포트가 3306
name = os.getenv('name')

# json 파일에서 불러오기
merged_json = "steamdb_indie_moba_merged.json"
df_merged = pd.read_json(merged_json, orient="records", encoding="utf-8")

# "oslist" 컬럼 제거 (업로드 시 포함하지 않음)
df_upload = df_merged.drop(columns=["oslist"])

# releaseYear 컬럼을 문자열로 변환
df_upload["releaseYear"] = df_upload["releaseYear"].astype(str)

# 데이터 확인: 상위 5개 항목과 info 출력
print("=== HEAD ===")
print(df_upload.head())
print("\n=== INFO ===")
print(df_upload.info())

# DB 연결 문자열 구성 (AWS의 MariaDB/MySQL) + charset=utf8mb4 추가
db_connection_str = f'mysql+pymysql://{dbuser}:{password}@{host}:{port}/{name}?charset=utf8mb4'
print("연결 문자열:", db_connection_str)

# 연결 시, init_command를 통해 문자셋 강제 설정
engine = create_engine(db_connection_str, connect_args={"init_command": "SET NAMES utf8mb4"})

# 테이블명: LIST_OF_MODA_INDI
table_name = "LIST_OF_MODA_INDI"

# dtype 매핑: 각 컬럼의 타입과 collation을 명시적으로 지정합니다.
dtype_mapping = {
    'name': VARCHAR(255, collation='utf8mb4_unicode_ci'),
    'app_id': BIGINT(),
    'price_us': FLOAT(),
    'releaseYear': VARCHAR(10, collation='utf8mb4_unicode_ci'),
    'userScore': FLOAT()
}

# 기존 테이블이 있다면 if_exists='replace'로 테이블 재생성하여 올바른 문자셋으로 생성
df_upload.to_sql(table_name, engine, if_exists='replace', index=False, dtype=dtype_mapping)
print(f"[DONE] '{table_name}' 테이블에 {len(df_upload)}개의 데이터가 업로드되었습니다.")



=== HEAD ===
                                           name   app_id  price_us  \
0  Bishojo Mangekyo Kotowari to Meikyu no Shojo  1310990     29.99   
1                                      MageQuit   572220     14.99   
2                                    Spellsworn   360620      0.00   
3                                          惊险极速  1777280      6.99   
4                    Awesomenauts - the 2D moba   204300      0.00   

  releaseYear  userScore  
0        2021      91.93  
1        2019      88.65  
2        2018      84.85  
3        2021      83.57  
4        2012      83.02  

=== INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         206 non-null    object 
 1   app_id       206 non-null    int64  
 2   price_us     206 non-null    float64
 3   releaseYear  206 non-null    object 
 4   userScore    149 non-null    